In [1]:
import numpy as np
import os
import torch
from model import *
from torch_geometric.data import Data, DataLoader
from scipy.sparse import coo_matrix
from utils import *
import random
from torch_geometric.loader import DataLoader as GeoDataLoader

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [25]:
# Set directory paths
dir_1 = './output_data/adjacency/adjacency_1'
dir_2 = './output_data/adjacency/adjacency_2'
dir_3 = './output_data/adjacency/adjacency_3'
dir_4 = './output_data/adjacency/adjacency_4'
dir_5 = './output_data/adjacency/adjacency_5'
dir_6 = './output_data/adjacency/adjacency_6'
dir_7 = './output_data/adjacency/adjacency_7'

image_dir = './Image_data/DATASET/train'

data_dict = {'1': [], '2': [], '3': [], '4': [], '5': [], '6': [], '7': []}

In [26]:
for filename in os.listdir(dir_1):
    if filename.endswith('.csv'):
        file_path = os.path.join(dir_1, filename)
        unique_id = filename.replace('adjacency_matrix_', '').replace('.csv', '')
        image_path = os.path.join(image_dir, f'1/{unique_id}')
        data = process_adj_matrix(file_path, image_path)
        data.y = torch.tensor([0])
        data_dict['1'].append(data)

for filename in os.listdir(dir_2):
    if filename.endswith('.csv'):
        file_path = os.path.join(dir_2, filename)
        unique_id = filename.replace('adjacency_matrix_', '').replace('.csv', '')
        image_path = os.path.join(image_dir, f'2/{unique_id}')
        data = process_adj_matrix(file_path, image_path)
        data.y = torch.tensor([1])
        data_dict['2'].append(data)

for filename in os.listdir(dir_3):
    if filename.endswith('.csv'):
        file_path = os.path.join(dir_3, filename)
        unique_id = filename.replace('adjacency_matrix_', '').replace('.csv', '')
        image_path = os.path.join(image_dir, f'3/{unique_id}')
        data = process_adj_matrix(file_path, image_path)
        data.y = torch.tensor([2])
        data_dict['3'].append(data)
        
for filename in os.listdir(dir_4):
    if filename.endswith('.csv'):
        file_path = os.path.join(dir_4, filename)
        unique_id = filename.replace('adjacency_matrix_', '').replace('.csv', '')
        image_path = os.path.join(image_dir, f'4/{unique_id}')
        data = process_adj_matrix(file_path, image_path)
        data.y = torch.tensor([2])
        data_dict['4'].append(data)

for filename in os.listdir(dir_5):
    if filename.endswith('.csv'):
        file_path = os.path.join(dir_5, filename)
        unique_id = filename.replace('adjacency_matrix_', '').replace('.csv', '')
        image_path = os.path.join(image_dir, f'5/{unique_id}')
        data = process_adj_matrix(file_path, image_path)
        data.y = torch.tensor([1])
        data_dict['5'].append(data)

for filename in os.listdir(dir_6):
    if filename.endswith('.csv'):
        file_path = os.path.join(dir_6, filename)
        unique_id = filename.replace('adjacency_matrix_', '').replace('.csv', '')
        image_path = os.path.join(image_dir, f'6/{unique_id}')
        data = process_adj_matrix(file_path, image_path)
        data.y = torch.tensor([2])
        data_dict['6'].append(data)
        
for filename in os.listdir(dir_7):
    if filename.endswith('.csv'):
        file_path = os.path.join(dir_7, filename)
        unique_id = filename.replace('adjacency_matrix_', '').replace('.csv', '')
        image_path = os.path.join(image_dir, f'7/{unique_id}')
        data = process_adj_matrix(file_path, image_path)
        data.y = torch.tensor([2])
        data_dict['7'].append(data)

In [33]:
data_list = data_dict['1'] + data_dict['2'] + data_dict['3'] + data_dict['4'] + data_dict['5'] + data_dict['6'] + data_dict['7']

batch_size = 64
loader = GeoDataLoader(data_list, batch_size=batch_size, shuffle=True)

num_node_features = data_list[0].x.shape[1]  # Number of node features
hidden_channels = 64

model = Net_Alex(num_node_features, hidden_channels).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epochs = 20

In [ ]:
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    correct = 0
    for batch in loader:
        batch.x = batch.x.to(device)
        batch.edge_index = batch.edge_index.to(device)
        batch.edge_weight = batch.edge_weight.to(device)
        batch.batch = batch.batch.to(device)
        batch.y = batch.y.to(device)
        batch.image_features = batch.image_features.to(device)  # Image tensor to device

        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.edge_weight, batch.batch, batch.image_features)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Compute accuracy
        _, pred = out.max(dim=1)
        correct += (pred == batch.y).sum().item()

    avg_loss = total_loss / len(loader)
    accuracy = correct / len(data_list)  # Overall accuracy
    print(f'Epoch {epoch}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')